# Face recognition

Face detection is based on 68 face landmark points like eye, brows, lips, nose, and mouth. It these landmarks points matches with the landmark of a general face then that can be considered as a "face". 

On the other hand, face detection relies on 128 points or face embeddings. These face embeddings are used to compare our face with the embeddings that we already have in the database.

In order to compare two faces perfectly, the face should be aligned. It should not be tilted or it should not be looking down or tilted to any side, so that it can compare with maximum efficiency. If not aligned , we have to align it by using different transformation techniques like translation, scaling, and rotation.

* Translation: image is shifting from a particular position to the next position without changing any height or widht.
* Scaling: image is changing the height or widht.
* Rotation: image is changing the angle/orientation of the image.

Once the alignment is complete the face will be perfectly okay for extracting the 128 features, which can call it as 128 dimensions of embeddings, which are different between image of different faces. 


## Implementation

#### Import libraries

In [1]:
import cv2
import face_recognition

### Image

In [ ]:
# Loading the image to detect
original_image = cv2.imread('../Images/Testing/Dufflo and Banerjee.jpg')

# Load the image to detect
dufflo_image = face_recognition.load_image_file("../Images/Sample/Dufflo.jpg")

# Extract face encodings. It returns a list of 128-dimensional face encodings.
dufflo_face_encoding = face_recognition.face_encodings(dufflo_image)[0]

banerjee_image = face_recognition.load_image_file("../Images/Sample/Banerjee.jpg")
banerjee_face_encoding = face_recognition.face_encodings(banerjee_image)[0]

# Create an array to save the encodings
known_face_encodings = [dufflo_face_encoding, banerjee_face_encoding]

# Create an array to hold labels
known_face_names = ["Dufflo", "Banerjee"]

# Load an unknown image to identify the faces
image_to_recognize = face_recognition.load_image_file("../Images/Testing/Dufflo and Banerjee.jpg")

# Find all the faces and encodings in the unknown image
all_face_locations = face_recognition.face_locations(image_to_recognize, number_of_times_to_upsample = 2, model = "hog")
all_face_encodings = face_recognition.face_encodings(image_to_recognize, all_face_locations)

# Print the number of faces detected
print("There are {} number of face in this image".format(len(all_face_locations)))

# Looping through the face locations and the face embeddings
for current_face_location, current_face_encoding in zip(all_face_locations, all_face_encodings):
    # Splitting the tuple to get the 4 position values of current face
    top_position, right_position, bottom_position, left_position = current_face_location
    # Find all the matches and get the list of matches
    all_matches = face_recognition.compare_faces(known_face_encodings, current_face_encoding)
    # String to hold the label
    name_of_person = 'Unknown face'
    # Check if the all matches have at least one item. If yes, get the index number of face that is located in the first index of all_matches
    if True in all_matches:
        first_match_index = all_matches.index(True)
        name_of_person = known_face_names[first_match_index]
    # Draw rectangle around the face detected
    cv2.rectangle(original_image, (left_position, top_position), (right_position, bottom_position), (255, 0, 0), 2)
    # Display the name
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(original_image, name_of_person, (left_position, bottom_position), font, 0.5, (255, 255, 255), 1)

    # Display the image
    cv2.imshow("Faces identified", original_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

### Video from real time webcam

In [15]:
# Capture the video from default camera
webcam_video_stream = cv2.VideoCapture(0)

# Load the sample images and get the 128 face embeddings from them
dufflo_image = face_recognition.load_image_file("../Images/Sample/Dufflo.jpg")
dufflo_face_encoding = face_recognition.face_encodings(dufflo_image)[0]

banerjee_image = face_recognition.load_image_file("../Images/Sample/Banerjee.jpg")
banerjee_face_encoding = face_recognition.face_encodings(banerjee_image)[0]

modi_image = face_recognition.load_image_file("../Images/Sample/Modi.jpg")
modi_face_encoding = face_recognition.face_encodings(modi_image)[0]

trump_image = face_recognition.load_image_file("../Images/Sample/Trump.jpg")
trump_face_encoding = face_recognition.face_encodings(trump_image)[0]

# Save the encodings and the corresponding laels in separate ararys in the same order
known_face_encodings = [dufflo_face_encoding, banerjee_face_encoding, modi_face_encoding, trump_face_encoding]
known_face_names = ["Dufflo", "Banerjee", "Modi", "Trump"]

# Initialize the array to hold all face locations, encodings and labels in the frame
all_face_locations = []
all_face_encodings = []
all_face_names = []

# Loop through every frame in the video
while True:
    # Get the current frame from the video stream as an image
    ret, current_frame = webcam_video_stream.read()
    # Resize the current frame to 1/4 size to proces faster
    current_frame_small = cv2.resize(current_frame, (0, 0), fx = 0.25, fy = 0.25)
    # Detect all faces in the image
    all_face_locations = face_recognition.face_locations(current_frame_small, 2, model = "hog")
    # Find all the faces and encodings in the unknown image
    all_face_encodings = face_recognition.face_encodings(current_frame_small, all_face_locations)

    # Looping through the face locations and the face embeddings
    for current_face_location, current_face_encoding in zip(all_face_locations, all_face_encodings):
        # Splitting the tuple to get the 4 position values of current face
        top_position, right_position, bottom_position, left_position = current_face_location
        # Change the position magnitude to fit the actual size video frame
        top_position = top_position * 4
        right_position = right_position * 4
        bottom_position = bottom_position * 4
        left_position = left_position * 4
        # Find all the matches and get the list of matches
        all_matches = face_recognition.compare_faces(known_face_encodings, current_face_encoding)
        # String to hold the label
        name_of_person = 'Unknown face'
        # Check if the all matches have at least one item. If yes, get the index number of face that is located in the first index of all_matches
        if True in all_matches:
            first_match_index = all_matches.index(True)
            name_of_person = known_face_names[first_match_index]
        # Draw rectangle around the face detected
        cv2.rectangle(current_frame, (left_position, top_position), (right_position, bottom_position), (255, 0, 0), 2)
        # Display the name
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(current_frame, name_of_person, (left_position, bottom_position), font, 0.5, (255, 255, 255), 1)

        # Display the image
        cv2.imshow("Webcam video", current_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the stream and cam
# Close all opencv windows open
webcam_video_stream.release()
cv2.destroyAllWindows()


There are 2 number of face in this image
